In [ ]:
class StackUsingList:
    def __init__(self):
        self.stack = []  # List as underlying storage
    
    def push(self, item):
        """Add item to top - O(1) amortized"""
        self.stack.append(item)
    
    def pop(self):
        """Remove and return top item - O(1)"""
        if self.is_empty():
            raise IndexError("Pop from empty stack")
        return self.stack.pop()
    
    def peek(self):
        """Return top item without removing - O(1)"""
        if self.is_empty():
            return None
        return self.stack[-1]
    
    def is_empty(self):
        """Check if stack is empty - O(1)"""
        return len(self.stack) == 0
    
    def size(self):
        """Return number of items - O(1)"""
        return len(self.stack)
    
    def display(self):
        """Display stack from top to bottom"""
        print("Top ->", end=" ")
        for item in reversed(self.stack):
            print(item, end=" -> ")
        print("Bottom")

# 3. Real-World Project 1: Undo/Redo System for Text Editor

In [ ]:
class TextOperation:
    def __init__(self, operation_type, position, text=None, old_text=None):
        self.operation_type = operation_type  # 'insert', 'delete', 'replace'
        self.position = position
        self.text = text  # Text that was inserted or new text for replace
        self.old_text = old_text  # Text that was deleted or old text for replace
    
    def __str__(self):
        if self.operation_type == 'insert':
            return f"Insert '{self.text}' at position {self.position}"
        elif self.operation_type == 'delete':
            return f"Delete '{self.old_text}' from position {self.position}"
        else:  # replace
            return f"Replace '{self.old_text}' with '{self.text}' at position {self.position}"


class TextEditor:
    def __init__(self):
        self.text = ""
        self.undo_stack = StackUsingList()  # For undoing operations
        self.redo_stack = StackUsingList()  # For redoing undone operations
        self.cursor_position = 0
    
    def insert_text(self, text, position=None):
        """Insert text at specified position"""
        if position is None:
            position = self.cursor_position
        
        # Save operation for undo
        operation = TextOperation('insert', position, text=text)
        self.undo_stack.push(operation)
        
        # Clear redo stack when new operation is performed
        self.redo_stack.stack.clear()
        
        # Perform insertion
        self.text = self.text[:position] + text + self.text[position:]
        self.cursor_position = position + len(text)
        
        print(f"Inserted '{text}' at position {position}")
    
    def delete_text(self, position, length):
        """Delete text starting from position"""
        if position >= len(self.text) or length <= 0:
            return
        
        # Ensure we don't delete beyond text length
        end_pos = min(position + length, len(self.text))
        deleted_text = self.text[position:end_pos]
        
        # Save operation for undo
        operation = TextOperation('delete', position, old_text=deleted_text)
        self.undo_stack.push(operation)
        
        # Clear redo stack
        self.redo_stack.stack.clear()
        
        # Perform deletion
        self.text = self.text[:position] + self.text[end_pos:]
        self.cursor_position = position
        
        print(f"Deleted '{deleted_text}' from position {position}")
    
    def replace_text(self, position, old_length, new_text):
        """Replace text at position with new text"""
        if position >= len(self.text):
            return
        
        # Ensure we don't replace beyond text length
        end_pos = min(position + old_length, len(self.text))
        old_text = self.text[position:end_pos]
        
        # Save operation for undo
        operation = TextOperation('replace', position, text=new_text, old_text=old_text)
        self.undo_stack.push(operation)
        
        # Clear redo stack
        self.redo_stack.stack.clear()
        
        # Perform replacement
        self.text = self.text[:position] + new_text + self.text[end_pos:]
        self.cursor_position = position + len(new_text)
        
        print(f"Replaced '{old_text}' with '{new_text}' at position {position}")
    
    def undo(self):
        """Undo the last operation"""
        if self.undo_stack.is_empty():
            print("Nothing to undo")
            return False
        
        operation = self.undo_stack.pop()
        
        # Perform reverse operation
        if operation.operation_type == 'insert':
            # Reverse of insert is delete
            self.text = self.text[:operation.position] + self.text[operation.position + len(operation.text):]
            reverse_op = TextOperation('delete', operation.position, old_text=operation.text)
            self.cursor_position = operation.position
        
        elif operation.operation_type == 'delete':
            # Reverse of delete is insert
            self.text = self.text[:operation.position] + operation.old_text + self.text[operation.position:]
            reverse_op = TextOperation('insert', operation.position, text=operation.old_text)
            self.cursor_position = operation.position + len(operation.old_text)
        
        else:  # replace
            # Reverse of replace is replace with old text
            new_text_length = len(operation.text)
            self.text = self.text[:operation.position] + operation.old_text + self.text[operation.position + new_text_length:]
            reverse_op = TextOperation('replace', operation.position, text=operation.old_text, old_text=operation.text)
            self.cursor_position = operation.position + len(operation.old_text)
        
        # Push to redo stack
        self.redo_stack.push(reverse_op)
        
        print(f"Undid: {operation}")
        return True
    
    def redo(self):
        """Redo the last undone operation"""
        if self.redo_stack.is_empty():
            print("Nothing to redo")
            return False
        
        operation = self.redo_stack.pop()
        
        # Perform the operation again
        if operation.operation_type == 'insert':
            self.text = self.text[:operation.position] + operation.text + self.text[operation.position:]
            self.cursor_position = operation.position + len(operation.text)
        
        elif operation.operation_type == 'delete':
            end_pos = operation.position + len(operation.old_text)
            self.text = self.text[:operation.position] + self.text[end_pos:]
            self.cursor_position = operation.position
        
        else:  # replace
            old_text_length = len(operation.old_text)
            self.text = self.text[:operation.position] + operation.text + self.text[operation.position + old_text_length:]
            self.cursor_position = operation.position + len(operation.text)
        
        # Push back to undo stack
        self.undo_stack.push(operation)
        
        print(f"Redid: {operation}")
        return True
    
    def get_text(self):
        """Get current text"""
        return self.text
    
    def get_cursor_position(self):
        """Get current cursor position"""
        return self.cursor_position
    
    def set_cursor_position(self, position):
        """Set cursor position"""
        self.cursor_position = min(max(0, position), len(self.text))
    
    def display_state(self):
        """Display current text and cursor position"""
        print("\n" + "="*60)
        print("TEXT EDITOR")
        print("="*60)
        print(f"Text: {self.text}")
        print(f"Cursor: {self.cursor_position}")
        
        # Show cursor position visually
        cursor_marker = " " * self.cursor_position + "|"
        print(f"        {cursor_marker}")
        
        print(f"Undo stack size: {self.undo_stack.size()}")
        print(f"Redo stack size: {self.redo_stack.size()}")
        print("="*60)


def text_editor_demo():
    editor = TextEditor()
    
    print("TEXT EDITOR WITH UNDO/REDO")
    print("-" * 40)
    
    # Insert some text
    editor.insert_text("Hello, ")
    editor.insert_text("world!")
    
    editor.display_state()
    
    # Replace text
    editor.replace_text(7, 6, "Python")
    editor.display_state()
    
    # Delete text
    editor.delete_text(12, 1)  # Delete '!'
    editor.display_state()
    
    # Undo operations
    print("\n=== Undoing ===")
    editor.undo()
    editor.display_state()
    
    editor.undo()
    editor.display_state()
    
    editor.undo()
    editor.display_state()
    
    # Redo operations
    print("\n=== Redoing ===")
    editor.redo()
    editor.display_state()
    
    editor.redo()
    editor.display_state()
    
    editor.redo()
    editor.display_state()
    
    # Try multiple undos/redos
    print("\n=== Multiple operations ===")
    editor.insert_text(" Programming")
    editor.replace_text(13, 11, " Development")
    editor.delete_text(0, 7)  # Remove "Hello, "
    
    editor.display_state()
    
    print("\n=== Undo all ===")
    while editor.undo():
        editor.display_state()
    
    print("\n=== Redo all ===")
    while editor.redo():
        editor.display_state()


if __name__ == "__main__":
    text_editor_demo()